In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

vgg_dir = 'tensorflow_vgg/'
# Make sure vgg exists
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(vgg_dir + "vgg16.npy"):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='VGG16 Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/vgg16.npy',
            vgg_dir + 'vgg16.npy',
            pbar.hook)
else:
    print("Parameter file already exists!") 

Parameter file already exists!


In [2]:
import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import glob

In [3]:
train_dir = 'data/train/'
train_kinds_ = glob.glob(train_dir + '*')

In [4]:
kinds = [each[11:] for each in train_kinds_ ]

In [13]:
for kind in kinds:
    paths = glob.glob(train_dir + kind + '/*')
    print(kind)
    print(len(paths))

melanoma
1100
nevus
1372
seborrheic_keratosis
254


### It is obvious that we should increase the number of the first one and the third one using opencv

In [11]:
import cv2
import numpy as np
from math import *
def increase(path,n=10):
    img = cv2.imread(path)
    img = cv2.resize(img,(224,224))
    height,width=img.shape[:2]
    degree=0
            #旋转后的尺寸
    for degree in range(0,n,5):
        heightNew=int(width*fabs(sin(radians(degree)))+height*fabs(cos(radians(degree))))
        widthNew=int(height*fabs(sin(radians(degree)))+width*fabs(cos(radians(degree))))

        matRotation=cv2.getRotationMatrix2D((width/2,height/2),degree,1)

        matRotation[0,2] +=(widthNew-width)/2  #重点在这步，目前不懂为什么加这步
        matRotation[1,2] +=(heightNew-height)/2  #重点在这步

        imgRotation=cv2.warpAffine(img,matRotation,(widthNew,heightNew),borderValue=(255,255,255))
        cv2.imwrite(path[:-4] + str(degree) + '.jpg' , imgRotation)
    

### increase

In [12]:
train_melanoma_dir = 'data/train/melanoma'
melanoma_paths = glob.glob(train_melanoma_dir + '/*')
print(len(melanoma_paths))
batch = 40
step = 0
for path in melanoma_paths:
    increase(path)
    step = step + 1
    if step % batch == 0:
        print(step)


405
40
80
120
160
200
240
280
320
360


KeyboardInterrupt: 

In [14]:
seborrheic_keratosis_dir = 'data/train/seborrheic_keratosis'
seborrheic_keratosis_paths = glob.glob(seborrheic_keratosis_dir + '/*')
print(len(seborrheic_keratosis_paths))
batch = 40
step = 0
for path in seborrheic_keratosis_paths:
    increase(path,n=15)
    step = step + 1
    if step % batch == 0:
        print(step)


254
40
80
120
160
200
240


### see it again?

In [15]:
for kind in kinds:
    paths = glob.glob(train_dir + kind + '/*')
    print(kind)
    print(len(paths))

melanoma
1100
nevus
1372
seborrheic_keratosis
1016


In [16]:
train_dir = 'data/train/'

In [ ]:
batch_size = 30
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)

    for each in kinds:
        print("Starting {} images".format(each))
        class_path = train_dir + kind
        files = glob.glob(class_path + '/*')
        for ii, file in enumerate(files, 1):
            # Add images to the current batch
            img = cv2.imread(file)
            # utils.load_image crops the input images for us, from the center
            img = cv2.resize(img,(224,224))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
            
            # Running the batch through the network to get the codes
            if ii % batch_size == 0 or ii == len(files):
                images = np.concatenate(batch)

                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                # Here I'm building an array of the codes
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                # Reset to start building the next batch
                batch = []
                print('{} images processed'.format(ii))

/home/carnd/dermatologist-ai/tensorflow_vgg/vgg16.npy
npy file loaded
build model started
build model finished: 0s
Starting melanoma images


In [ ]:
with open('codes_train', 'w') as f:
    codes.tofile(f)
    
# write labels to file
import csv
with open('labels_train', 'w') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(labels)